In [5]:
%run "../scripts/data_processing.py"
%run "../scripts/node_edge_generation.py"
%run "../scripts/graph_functions.py"

In [6]:
from pyteomics import fasta
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import json
import os
import subprocess
import csv
from neo4j import GraphDatabase

In [7]:
#Data path
datadir = os.path.abspath("../../Data/")

#Datasets
OT_targets    = datadir+"/OpenTargets/targets" #OpenTargets | "targets" data
OT_directevi  = datadir+"/OpenTargets/associationByOverallDirect" #OpenTargets | Disease association - Direct evidence data
OT_indireevi  = datadir+"/OpenTargets/associationByOverallIndirect" #OpenTargets | Disease association - Indirect evidence data
diseasedb     = datadir+"/OpenTargets/Disease" #OpenTargets "Disease" intel

Uniprot_hsap  = datadir+"/Uniprotdb/human_proteome/UP000005640_9606.fasta" #Uniprot h. sapiens proteome
Unisprot      = datadir+"/Uniprotdb/uniprot_sprot_human.dat" #Uniprot core dataset, contains functional intel
Genemap       = datadir+"/Uniprotdb/HUMAN_9606_idmapping.dat" #Uniprot gene ID mapping

Orpha_en      = datadir+"/Disease_ontology/orpha_en_product1.json" #Orphanet rare disease xef

# Functions

## Data input

In [ ]:
def genUniprot(unidata, genein):
    #Generate Uniprot <> Gene dictionary
    with open(genein) as f:
        read = f.readlines()
    unigene = {}
    for i in read:
        splitr = i.split("\t")
        if splitr[1] == "Gene_Name" or splitr[1] == "Gene_Synonym":
            unigene.setdefault(splitr[0], [])
            unigene[splitr[0]].append(splitr[2].strip("\n"))

    #Generate df of human proteome & dic of gene names to uniprot ids
    unidic    = {}
    fastain   = fasta.UniProt(unidata)
    count = 0
    for uni in fastain:
        count += 1
        unid = uni[0]['id']
        name = uni[0]['name']
        try:
            gene = unigene[unid][0]
        except:
            gene = uni[0]['gene_id']
        seq  = uni[1]
        unidic.setdefault(unid, {"Uniprot": unid, "Gene": gene, "Name": name, "Seq": seq})

    unidf = pd.DataFrame.from_dict(unidic, orient='index')

    return unidf

def parseOTtarget(data_dir, df):
    #Parse OpenTargets "targets" intel, return df
    #Add new intel cols to df
    intelcol = {"Ensembl_gene": 'id', \
                "Ensembl_trans": 'transcriptIds', \
                "Description": "functionDescriptions", \
                "Subcell_loc": "subcellularLocations", \
                "DB_entries": "dbXrefs", \
                "Bio_path": "pathways", \
                "Tx_approaches": "tractability"}
    for col in intelcol:
        df[col] = [list() for x in range(len(df.index))]
    
    data = []
    #Read in multi part .json to data list 
    for file in os.listdir(data_dir):
        path = data_dir + "/" + str(file).replace("._", "")
        with open(path, "r") as f:
            for line in f:
                data.append(json.loads(line))
                
    #Retrieve intel from protein coding entries, annotate df
    ensemtouni = {}
    for entry in data:
        try:
            entid = entry['proteinIds'][0]['id']
        except:
            entid = ""

        if entid in df.index:
            for j in intelcol:
               #if intelcol[j] in entry and j != "Bio_path":
                if intelcol[j] in entry and j not in ["Bio_path", "Subcell_loc"]:
                    df.loc[entid, j].append(entry[intelcol[j]])
                #Simplify biological pathways for easier querying
                elif intelcol[j] in entry and j == "Bio_path":
                    for path in entry[intelcol[j]]:
                        df.loc[entid, j].append([path['pathwayId'], path['pathway']])
                elif j == "Ensembl_gene":
                    ensemtouni.setdefault(entry[intelcol[j]], entid) 
                elif intelcol[j] in entry and j == "Subcell_loc":
                    allsubloc = []
                    if len(entry[intelcol[j]]) > 0:
                        for sub in entry[intelcol[j]]:
                            allsubloc.append(sub["location"].lower())   
                    df.loc[entid, j].append(allsubloc[0:])
                   # df.loc[entid, j] = allsubloc
                    
    
    return df, ensemtouni

def parseOTdisease(dataloc):
    #Parse OT disease <> protein evidence
    data = {}
    for file in os.listdir(dataloc):
        if ".json" in str(file):
            with open(dataloc + "/" + str(file).replace("._", "")) as f:
                for line in f.readlines():
                    linedict = json.loads(line)
                    data.setdefault(linedict["diseaseId"], [linedict])
                    data[linedict["diseaseId"]].append(linedict)
    return data

def findDisease(database):
    #Parse high level OT disease intel
    data = {}
    count = 0
    for file in os.listdir(database):
        if ".json" in str(file):
            count += 1
            try:
                with open(database + "/" + str(file)) as f:
                    for line in f.readlines():
                        linedict = json.loads(line)
                        data.setdefault(linedict["id"], linedict)
            except:
                pass

    return data

def funcGen(unisprot):
    #Return a dictionary of Uniprot to function annotations
    outdic = {}
    with open(unisprot, "r") as f:
        infile = f.read().replace(";", " ")
        splitin= infile.split("//\n")
    #Iterate per entry, split block, retrieve Uniprot + individual functions
    for uni in splitin:
        count = 0
        allfunc = [line for line in uni.split("\n") if line.startswith('FT')]
        allac = [line for line in uni.split("\n") if line.startswith('AC')]
        try:
            uniprot = re.search(r"\bAC\s+([A-Za-z0-9]+)", uni).group().split()[1]
        except:
            uniprot = ""
        funcdic = {}
        func = {}
        curannot = ""
        #Iterate over functions, append to out dic
        for i in allfunc:
            match = re.match(r"^\S+(\s+)\S+", i)
            if len(match.group(1)) == 3:
                funcdic.setdefault(count, func)
                curannot = ""
                spliti = i.split()
                count += 1
                nums = re.findall(r'\d+', spliti[-1])
                nums = [int(num) for num in nums]
                func = {"type": spliti[1].replace(";", " "), "range": nums, "note": "", "evidence": "", "id": ""}
            else:
                if re.search(r"(?<=\/)\w+=+", i) != None:
                    search = re.search(r"(?<=\/)\w+=+", i)
                    curannot = search.group()[:-1]
                    func[curannot] = [i.split("=")[1].replace("\"", "")] 
                    spliti = i.split("/id=")
                else:
                    func[curannot] = func[curannot][0]+" ".join(i.split()[1:])
        outdic.setdefault(uniprot, funcdic)
    return outdic

def parseOrpha(datain):
    ##TO-DO
        #Alter output to df with cols = ontologies    
    #Read in Orphanet json, return dataframe of Orpha code / Name / other database crossreferences
    with open(datain) as f:
        dataj = json.load(f)
    outdic = {}
    for i in dataj["JDBOR"][0]["DisorderList"][0]["Disorder"]:
        orpha = i["OrphaCode"]
        nam   = i["Name"][0]["label"]
        xrefs = {}
        try:
            for j in i["ExternalReferenceList"][0]['ExternalReference']:
                xrefs.setdefault(j['Source'], {"ID": j['id'], "Mapping": j['DisorderMappingRelation'][0]["Name"][0]["label"]})
        except:
            pass
        outdic.setdefault(orpha, {"Name": nam, "Xref": xrefs})

    outdf = pd.DataFrame.from_dict(outdic)

    return outdf.transpose() 

def assocUniprotDisease(datadic, ensgenetouni, diseasexref):
    #Generate dictionary linking Uniprot ID to disease associations + scores
    uniprotdisease = {}
    for i in datadic:
        for ent in datadic[i]:
            try:
                Orpha   = int(ent['diseaseId'].split("_")[1])
                Score   = round(ent['score'], 3)
                Uniprot = ensgenetouni[ent['targetId']]
                Disnam  = diseasexref.loc[str(Orpha)]["Name"]
                uniprotdisease.setdefault(Uniprot, {Orpha: {"Disease": Disnam, "Score": Score}})
                uniprotdisease[Uniprot][Orpha] = {"Disease": Disnam, "Score": Score}
            except:
                pass
    return uniprotdisease

## Node / Edge generation

In [ ]:
def genProtNode(df):
    #Generate Uniprot Protein node
    count = 0
    outcsv = [["id:ID", ":LABEL", "uniprot", "gene", "name", "seq", "ens_gene", "ens_trans", "description", "db_entries", "bio_path", "tx_approach", "subcellular_location"]]
    nodecodec = {}
    for index, row in df.iterrows():
        count += 1
        outcsv.append(["P"+str(count), "Protein", row["Uniprot"], row['Gene'], row['Name'], row['Seq'], row['Ensembl_gene'], row['Ensembl_trans'],\
                        row['Description'], row['DB_entries'], row['Bio_path'], row['Tx_approaches'], row["Subcell_loc"]])
        nodecodec.setdefault(index, "P"+str(count))
    return outcsv, nodecodec

def genFuncNode(funcdic):
    #Generate Uniprot Function node
    count = 0
    outcsv = [["id:ID", ":LABEL", "type",  "range", "alt_id", "evidence", "note"]]
    nodecodec = {}
    for uni in funcdic:
        for func in funcdic[uni]: 
            if len(funcdic[uni][func]) > 0:
                count += 1
                outcsv.append(["F"+str(count), "Function", funcdic[uni][func]["type"], funcdic[uni][func]["range"], funcdic[uni][func]["id"], funcdic[uni][func]["evidence"], funcdic[uni][func]["note"]])
                nodecodec.setdefault("F"+str(count), uni)
    return outcsv, nodecodec

def genDisNode(disdic, disedge):
    #Generate OpenTarget disease node
    count = 0
    outcsv = [["id:ID", ":LABEL", "id_ont", "name", "description"]]
    nodecodec = {}
    for dis in disdic:
        if dis in disedge:
            try:
                desc = disdic[dis]["description"]
            except:
                desc = ""
            count += 1
            outcsv.append(["D"+str(count), "Disease", dis, disdic[dis]["name"], desc.replace("\n", " ")])
            nodecodec.setdefault(dis, "D"+str(count))
    return outcsv, nodecodec

def genProtFuncEdge(funcod, protcod):
    #Generate Protein <> Functional edges
    outcsv = [[":START_ID",":END_ID", ":TYPE"]]
    count = 0
    for i in funcod:
        if funcod[i] in protcod:
            outcsv.append([i, protcod[funcod[i]], "is_feature"])

    return outcsv

def genDisProtEdge(disdic, protcod, discod, ensdic):
    #Generate Disease <> Functional edges
    outcsv = [[":START_ID",":END_ID", ":TYPE", "score"]]
    for i in disdic:
        for j in disdic[i]:
            if j["targetId"] in ensdic:
                outcsv.append([protcod[ensdic[j["targetId"]]], discod[i], "direct_evidence", float(j["score"])])
    return outcsv

## Graph read / write

In [ ]:
def nodeToCSVprep(genenode, diseasenode):
    #Generate out lists for .csv out
    geneout = [["id:ID", ":LABEL", "name", "ensemble_id"]]
    for i in genenode:
        geneout.append([genenode[i]["ID"], "Gene", i, genenode[i]["Ensemble"]])
        
    disout = [["id:ID", ":LABEL", "name", "uri"]]
    for i in diseasenode:
        disout.append([diseasenode[i]["ID"], "Disease", i, diseasenode[i]["URI"]])
        
    return geneout, disout

def writecsv(infile, outfile):
    #Write out csv
    with open(outfile, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerows(infile)

def run_query(query):
    #Submit query to graph database, retrieve output as df
    with driver.session() as session:
        result = session.run(query)
        rows = [record.data() for record in result]
        df = pd.DataFrame(rows)
        return df

# Main script

## Data prep

In [9]:
#Read in Uniprot proteome intel
start = time.time()
unidf = genUniprot(Uniprot_hsap, Genemap)
print("Time taken | Uniprot initial proteome = ", round(time.time() - start, 2))

#Read in OT target intel - DONE
start = time.time()
unidf, ensemtouni = parseOTtarget(OT_targets, unidf)
print("Time taken | OpenTargets targets intel = ", round(time.time() - start, 2))

#Lookup dic of ensemble gene ids to uniprot
start = time.time()
ensgenetouni = {}
genetouni    = {}
for index, row in unidf.iterrows():
    val = row["Ensembl_gene"]
   # print(val)
    for i in row["Ensembl_gene"]:
        ensgenetouni.setdefault(i.replace("\'", ""), index)
    genetouni.setdefault(row["Gene"], index)
print("Time taken | Ensemble <> Uniprot | Gene <> Uniprot lookup dic = ", round(time.time() - start, 2))

#Read in OT functional intel 
start = time.time()
funcdic = funcGen(Unisprot)
print("Time taken | Uniprot function intel = ", round(time.time() - start, 2))

#Read in OT disease intel
start = time.time()
diseasego = findDisease(diseasedb) 
print("Time taken | OpenTargets init disease intel = ", round(time.time() - start, 2))

#Read in disease associations
start = time.time()
OT_direct = parseOTdisease(OT_directevi)
OT_indir  = parseOTdisease(OT_indireevi)
print("Time taken | OpenTargets dir/ind disease intel = ", round(time.time() - start, 2))

#Generate rare disease crossreference
start = time.time()
diseasexref = parseOrpha(Orpha_en)
print("Time taken | Rare disese xref = ", round(time.time() - start, 2))

#Generate dictionaries of Uniprot to disease assocations
start = time.time()
dirdic = assocUniprotDisease(OT_direct, ensgenetouni, diseasexref)
inddic = assocUniprotDisease(OT_indir, ensgenetouni, diseasexref)
print("Time taken | Uniprot disease association = ", round(time.time() - start, 2))

Time taken | Uniprot initial proteome =  1.35
Time taken | OpenTargets targets intel =  7.76
Time taken | Ensemble <> Uniprot | Gene <> Uniprot lookup dic =  0.32
Time taken | Uniprot function intel =  8.57
Time taken | OpenTargets init disease intel =  0.21
Time taken | OpenTargets dir/ind disease intel =  18.5
Time taken | Rare disese xref =  1.1
Time taken | Uniprot disease association =  65.88


## Node / Edge generation

In [10]:
#Generate nodes / edges csv
protnodes, protcodec = genProtNode(unidf)
funcnodes, funccodec = genFuncNode(funcdic)
disnodes, discodec   = genDisNode(diseasego, OT_direct)
profuncedge          = genProtFuncEdge(funccodec, protcodec)
disprotedge          = genDisProtEdge(OT_direct, protcodec, discodec, ensgenetouni)
#transnod, transedge  = genscRNANodeEdge(scdf, genetouni, protcodec)

In [ ]:
#Write out node + edge csv files
if os.path.exists("./Results") == False:
    os.mkdir("./Results")

#Write out nodes / edges csv
writecsv(protnodes, "./Results/protnodes.csv")
writecsv(disnodes, "./Results/disnodes.csv")
writecsv(funcnodes, "./Results/funcnodes.csv")
writecsv(profuncedge, "./Results/profuncedge.csv")
writecsv(disprotedge, "./Results/disprotedge.csv")

## Start graph

In [ ]:
def startGraph(passloc, directory):
    #Password
    with open(passloc, "r") as f:
        password = f.read()
    
    curdir = os.path.abspath(directory)
    #Commands for stopping / generating / starting neo4j
    cmd0 = ["sudo", "-S", "neo4j", "stop"]
    cmd1 = [ \
            "sudo", "-S", "neo4j-admin", "database", "import", "full", "neo4j", \
            "--nodes="+ curdir+"/protnodes.csv", \
            "--nodes="+ curdir+"/disnodes.csv", \
            "--nodes="+ curdir+"/funcnodes.csv", \
            "--relationships="+ curdir+"/profuncedge.csv", \
            "--relationships="+ curdir+"/disprotedge.csv", \
            "--delimiter=;", \
            "--array-delimiter=|", \
            "--overwrite-destination", \
            "--verbose"
            ]
    cmd2 = ["sudo", "-S", "neo4j", "start", "--verbose"]
    
    #Stop server
    result = subprocess.run(cmd0, input=password+"\n", capture_output=True, text=True)
    if result.returncode == 0:
        print("Graph stopped")
    else:
        print(f"Stop failed with error:\n{result.stderr}")
    
    #Generate graph
    result = subprocess.run(cmd1, input=password+"\n", capture_output=True, text=True)
    if result.returncode == 0:
        print("Database import successful!")
    else:
        print(f"Import failed with error:\n{result.stderr}")
    
    #Start graph
    result = subprocess.run(cmd2, input=password+"\n", capture_output=True, text=True)
    if result.returncode == 0:
        print("Graph started")
    else:
        print(f"Start failed with error:\n{result.stderr}")


startGraph("./pass_ent.txt", "./Results/")

In [5]:
startGraph("./pass_ent.txt", "./Results/")

Graph stopped
Database import successful!
Graph started


## Run cypher

In [11]:
uri = "bolt://localhost:7687" 
driver = GraphDatabase.driver(uri)

#“Which traits are associated with gene $gene’s_name?”
query_1 = [f'match (p:Protein)\n \
            where not p.subcellular_location contains "[]" \
            return p.uniprot as Uniprot, p.subcellular_location as Subcellular_location, p.seq as Sequence']
#print(f"\nWhich traits are associated with gene {gene_name}?")
results_1 = run_query(query_1[0], driver)
display(results_1)

#Close connection
driver.close()

,Uniprot,Subcellular_location,Sequence
0,A2RRL7,[['membrane ; single-pass type i membrane prot...,MQRLPAATRATLILSLAFASLHSACSAEASSSNSSSLTAHHPDPGT...
1,A8MYZ6,"[['cytoplasm', 'nucleus']]",MAAKLRAHQVDVDPDFAPQSRPRSCTWPLPQPDLAGDEDGALGAGV...
2,O14804,[['cell membrane ; multi-pass membrane protein']],MRAVFIQGAEEHPAAFCYQVNGSCPRTVHTLGIQLVIYLACAAGML...
3,O75459,"[['nucleoli fibrillar center', 'mitochondria',...",MGFLRRLIYRRRPMIYVESSEESSDEQPDEVESPTQSQDSTPAEER...
4,O75912,"[['cell projection, axon', 'cell projection, d...",MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...
...,...,...,...
18417,Q16526,"[['cytoplasm', 'nucleus', 'microtubules', 'nuc...",MGVNAVHWFRKGLRLHDNPALKECIQGADTIRCVYILDPWFAGSSN...
18418,Q3B7T1,"[['nucleus', 'nucleoplasm', 'centrosome', 'cyt...",MGDAKEAGAEGPPAGAAARGGLSLLSQGESEESSAQGSALFLGGNE...
18419,Q499Z4,"[['nucleus', 'nucleoplasm']]",MFATSGAVAAGKPYSCSECGKSFCYSSVLLRHERAHGGDGRFRCLE...
18420,Q5H8A4,[['endoplasmic reticulum membrane ; multi-pass...,MRLGSGTFATCCVAIEVLGIAVFLRGFFPAPVRSSARAEHGAEPPA...


In [ ]:
call {match (d:Disease)-[de1:direct_evidence]-(p1:Protein)
where toFloat(de1.score) > 0.6
return p1, de1, d}
match (d)-[de2]-(p2:Protein)-[ife1:is_feature]-(f:Function)
where toLower(f.note) contains toLower(p1.gene)
and not p1.gene = p2.gene
and toFloat(de2.score) > 0.6
return p2.gene as Gene_1, p1.gene as Gene_2, d.name as Disease, collect(distinct f.note) as Function 
order by size(Function) desc limit 50 

# Function improvements

In [23]:
unidf

,Uniprot,Gene,Name,Seq,Ensembl_gene,Ensembl_trans,Description,Subcell_loc,DB_entries,Bio_path,Tx_approaches
A0A075B6U7,A0A075B6U7,TRAJ23,T cell receptor alpha joining 23 (Fragment),XIYNQGGKLIFGQGTELSVKP,[ENSG00000211866],[[ENST00000390514]],[],[],"[[{'id': '12052', 'source': 'HGNC'}]]",[],"[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
A0A2R8YED5,A0A2R8YED5,OR5BS1P,Olfactory receptor,MEVSNMTTVTVFILLGLSNNPQVQALLFVLFLVIYLLTLLGNLLMV...,[ENSG00000198678],[[ENST00000328207]],[],[],"[[{'id': '19627', 'source': 'HGNC'}]]",[],"[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
A2RRL7,A2RRL7,TMEM213,Transmembrane protein 213,MQRLPAATRATLILSLAFASLHSACSAEASSSNSSSLTAHHPDPGT...,[ENSG00000214128],"[[ENST00000458494, ENST00000442682, ENST000004...",[[]],[[membrane ; single-pass type i membrane prote...,"[[{'id': '27220', 'source': 'HGNC'}, {'id': 'I...",[],"[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
A6NHS1,A6NHS1,YK042,Putative uncharacterized protein ENSP00000347057,MVLLAGTRPQGGEARCMIPPPPSPLLGAQVEEDRTEFKEFQDFSSL...,[],[],[],[],[],[],[]
A6NL46,A6NL46,YF016,Putative UPF0607 protein ENSP00000332738,MRLCLIPWNTTPHRVLPPVVWSAPSRKKPVLSARNSMMFGHLSPVR...,[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
Q9UPU3,Q9UPU3,SORCS3,VPS10 domain-containing receptor SorCS3,MEAARTERPAGRPGAPLVRTGLLLLSTWVLAGAEITWDATGGPGRP...,[ENSG00000156395],"[[ENST00000393176, ENST00000369701]]",[[]],[[membrane; single-pass type i membrane protein]],"[[{'id': '16699', 'source': 'HGNC'}, {'id': 'I...",[],"[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
Q9Y2Z9,Q9Y2Z9,COQ6,"Ubiquinone biosynthesis monooxygenase COQ6, mi...",MAARLVSRCGAVRAAPHSGPLVSWRRWSGASTDTVYDVVVSGGGLV...,[ENSG00000119723],"[[ENST00000553922, ENST00000557780, ENST000003...",[[FAD-dependent monooxygenase required for the...,[[mitochondrion inner membrane ; peripheral me...,"[[{'id': '20233', 'source': 'HGNC'}, {'id': 'R...","[[R-HSA-2142789, Ubiquinol biosynthesis]]","[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
Q9Y328,Q9Y328,NSG2,Neuronal vesicle trafficking-associated protein 2,MVKLNSNPSEKGTKPPSVEDGFQTVPLITPLEVNHLQLPAPEKVIV...,[ENSG00000170091],"[[ENST00000517902, ENST00000517587, ENST000005...",[[]],[[membrane ; single- pass type ii membrane pro...,"[[{'id': '24955', 'source': 'HGNC'}, {'id': 'I...",[],"[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."
Q9Y5S2,Q9Y5S2,CDC42BPB,Serine/threonine-protein kinase MRCK beta,MSAKVRLKKLEQLLLDGPWRNESALSVETLLDVLVCLYTECSHSAL...,[ENSG00000198752],"[[ENST00000559790, ENST00000558321, ENST000005...",[[Serine/threonine-protein kinase which is an ...,"[[cytoplasm, cell membrane ; peripheral membra...","[[{'id': '1738', 'source': 'HGNC'}, {'id': '3Q...","[[R-HSA-9013409, RHOJ GTPase cycle], [R-HSA-90...","[[{'modality': 'SM', 'id': 'Approved Drug', 'v..."


In [43]:
def genSubcellNode(df):
    #Subcellular localisation annotations for broad categories
    uniloc = {"Intracellular": [], "Extracellular": [], "Membrane": []}

    #Identify unique locations
    indiv_loc = {}
    for loc_list in df["Subcell_loc"].values:
        for ent in loc_list:
            for loc in ent:
                if loc not in indiv_loc:
                    indiv_loc.setdefault(loc, "")
                    
    #Add broad subcellular locations based on keywords
    for loc in indiv_loc:
        if any( i in loc.lower().split() for i in ["secreted", "secret", "extracellular"]):
            indiv_loc[loc] = "extracellular"
        else:
            if any( i in loc.lower().split() for i in ["membrane", "wall", "lipid"]):
                indiv_loc[loc] = "membrane"
            else:
                indiv_loc[loc] = "intracellular"
    
    #Generate Subcellular Location node
    count = 0
    outcsv = [["id:ID", ":LABEL", "location", "intra_mem_extra"]]
    nodecodec = {}
    for loc in indiv_loc:
        count += 1
        outcsv.append(["S"+str(count), "Subcellular", loc, indiv_loc[loc]])
        nodecodec.setdefault(loc, "S"+str(count))

    return outcsv, nodecodec
    
csvtest, codectest = genSubcellNode(unidf)

for i in codectest:
    print(codectest[i], i)


S1 membrane ; single-pass type i membrane protein
S2 cytoplasm
S3 nucleus
S4 cell membrane ; multi-pass membrane protein
S5 nucleoli fibrillar center
S6 mitochondria
S7 nucleoplasm
S8 cell projection, axon
S9 cell projection, dendrite
S10 cell junction, synapse, presynapse
S11 cell junction, synapse, postsynapse
S12 cell junction, synapse, postsynaptic density
S13 cell junction, synapse, synaptic cell membrane
S14 cytoplasmic vesicle, secretory vesicle, synaptic vesicle membrane
S15 cytoplasm, cytosol
S16 cytoplasmic vesicle, copii-coated vesicle membrane ; peripheral membrane protein ; cytoplasmic side
S17 endoplasmic reticulum membrane ; peripheral membrane protein ; cytoplasmic side
S18 vesicles
S19 microsome membrane ; single-pass type iii membrane protein
S20 endoplasmic reticulum membrane ; single-pass type iii membrane protein
S21 membrane ; multi-pass membrane protein
S22 cytosol
S23 plasma membrane
S24 intermediate filaments
S25 [isoform mitochondrial]: mitochondrion
S26 [isof

In [11]:
def parseOTtarget2(data_dir, df):
    #Parse OpenTargets "targets" intel, return df
    #Add new intel cols to df
    intelcol = {"Ensembl_gene": 'id', \
                "Ensembl_trans": 'transcriptIds', \
                "Description": "functionDescriptions", \
                "Subcell_loc": "subcellularLocations", \
                "DB_entries": "dbXrefs", \
                "Bio_path": "pathways", \
                "Tx_approaches": "tractability"}
    for col in intelcol:
        df[col] = [list() for x in range(len(df.index))]
    
    data = []
    #Read in multi part .json to data list 
    for file in os.listdir(data_dir):
        path = data_dir + "/" + str(file).replace("._", "")
        with open(path, "r") as f:
            for line in f:
                data.append(json.loads(line))
                
    #Retrieve intel from protein coding entries, annotate df
    ensemtouni = {}
    for entry in data:
        try:
            entid = entry['proteinIds'][0]['id']
        except:
            entid = ""

        if entid in df.index:
            for j in intelcol:
               #if intelcol[j] in entry and j != "Bio_path":
                if intelcol[j] in entry and j not in ["Bio_path", "Subcell_loc"]:
                    df.loc[entid, j].append(entry[intelcol[j]])
                #Simplify biological pathways for easier querying
                elif intelcol[j] in entry and j == "Bio_path":
                    for path in entry[intelcol[j]]:
                        df.loc[entid, j].append([path['pathwayId'], path['pathway']])
                elif j == "Ensembl_gene":
                    ensemtouni.setdefault(entry[intelcol[j]], entid) 
                elif intelcol[j] in entry and j == "Subcell_loc":
                    allsubloc = []
                    print(entry[intelcol[j]])
                    if len(entry[intelcol[j]]) > 0:
                        for sub in entry[intelcol[j]]:
                            allsubloc.append(sub["location"].lower())   
                    df.loc[entid, j].append(allsubloc[0:])
                   # df.loc[entid, j] = allsubloc
                    
    
    return df, ensemtouni

unidf2, ensemtouni2 = parseOTtarget2(OT_targets, unidf)

[{'location': 'Endoplasmic reticulum membrane ; Single-pass type II membrane protein', 'source': 'uniprot'}, {'location': 'Golgi apparatus', 'source': 'uniprot'}]
[{'location': 'Secreted', 'source': 'uniprot'}]
[{'location': 'Cytoplasm', 'source': 'uniprot'}, {'location': 'Nucleus', 'source': 'uniprot'}, {'location': 'Nucleoplasm', 'source': 'HPA_main'}, {'location': 'Centrosome', 'source': 'HPA_additional'}, {'location': 'Vesicles', 'source': 'HPA_additional'}]
[{'location': 'Nucleus, nucleolus', 'source': 'uniprot'}, {'location': 'Nucleus membrane ; Single-pass membrane protein', 'source': 'uniprot'}, {'location': 'Nucleoli', 'source': 'HPA_main'}, {'location': 'Cytosol', 'source': 'HPA_additional'}, {'location': 'Plasma membrane', 'source': 'HPA_additional'}, {'location': 'Vesicles', 'source': 'HPA_additional'}]
[{'location': 'Cytoplasm', 'source': 'uniprot'}, {'location': 'Cytosol', 'source': 'HPA_main'}, {'location': 'Plasma membrane', 'source': 'HPA_additional'}]
[{'location': 'C

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[{'location': 'Endomembrane system ; Peripheral membrane protein', 'source': 'uniprot'}, {'location': 'Nucleoli fibrillar center', 'source': 'HPA_main'}, {'location': 'Vesicles', 'source': 'HPA_main'}]
[{'location': 'Cytosol', 'source': 'HPA_main'}, {'location': 'Cytokinetic bridge', 'source': 'HPA_additional'}]
[{'location': 'Secreted', 'source': 'uniprot'}]
[{'location': 'Cytoplasm', 'source': 'uniprot'}, {'location': 'Cell membrane', 'source': 'uniprot'}, {'location': 'Cytosol', 'source': 'HPA_main'}]
[{'location': 'Cell membrane ; Multi-pass membrane protein', 'source': 'uniprot'}, {'location': 'Basolateral cell membrane ; Multi-pass membrane protein', 'source': 'uniprot'}, {'location': 'Plasma membrane', 'source': 'HPA_main'}, {'location': 'Nucleoplasm', 'source': 'HPA_additional'}]
[{'location': 'Mitochondrion outer membrane', 'source': 'uniprot'}, {'location': 'Membrane', 'source': 'uniprot'}, {'location': 'Mitochondria', 'source': 'HPA_main'}]
[{'location': 'Cytoplasm', 'source